In [ ]:
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt

4(a)

In [ ]:
text_path = '/content/drive/MyDrive/5460/book-war-and-peace.txt'

In [ ]:
f = open(text_path)
content = []
for line in f:
  content.append(line.strip())

In [ ]:
content = ''.join(content)

In [ ]:
char_list = sorted(list(set(content)))
length = len(char_list)
length

81

In [ ]:
token_index = {}
char_index = {}
for index, char in enumerate(char_list):
  token_index[char] = index
  char_index[index] = char

In [ ]:
def vector(text):
  vector = np.zeros([len(text), length])
  index = np.zeros(len(text))
  for i, char in enumerate(text):
    vector[i, token_index[char]] = 1
    index[i] = token_index[char]
  return torch.Tensor(vector), torch.Tensor(index).long()

In [ ]:
content_vector, index = vector(content)
content_vector, index

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([27, 32, 25,  ..., 71, 69,  9]))

4(b)

In [ ]:
is_cuda = torch.cuda.is_available()

if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [ ]:
# parameters
input_size = length
output_size = length
hidden_dim = 200
num_layers = 1

# creat training and validation data set

percent = 0.8

X_train = content_vector[:int(percent*len(content_vector))]
X_val = content_vector[int(percent*len(content_vector)):]
y_train = index[:int(percent*len(content_vector))]
y_val = index[int(percent*len(content_vector)):]

# minus 1 here makes sure y also has the dimension of 25
train_batch_num = (len(X_train)-1)//25
val_batch_num = (len(X_val)-1)//25

def batch_generator(i, x, y):
  x_out = x[i*25:(i+1)*25].to(device)
  y_out = y[i*25+1:(i+1)*25+1].to(device)
  return x_out, y_out

train_batch_num, val_batch_num

(100430, 25107)

In [ ]:
class RNN(nn.Module):
  def __init__(self, input_size, output_size, hidden_dim, num_layers):
    super(RNN, self).__init__()

    self.hidden_dim = hidden_dim
    self.num_layers = num_layers

    self.rnn = nn.RNN(
        input_size = input_size, 
        hidden_size = hidden_dim,
        num_layers = num_layers,
        nonlinearity = 'tanh'
    )
    self.linear1 = nn.Linear(input_size, input_size)
    self.linear2 = nn.Linear(hidden_dim, output_size)
  
  def forward(self, x, hidden):
    x = self.linear1(x)
    x = x.view(-1, 1, input_size)
    output, h_n = self.rnn(x, hidden)
    output = output.squeeze(1)
    output = self.linear2(output)
    return output, h_n

net = RNN(input_size, output_size, hidden_dim, num_layers).to(device)

In [ ]:
epochs = 5
learning_rate = 1e-3
batch_size = 1

optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
train_loss_list = []
val_loss_list = []

hidden = torch.zeros(num_layers, batch_size, hidden_dim, requires_grad=True).to(device)
j = 0

for epoch in range(1,epochs+1):
  train_loss = 0
  for i in range(train_batch_num):
    net.train()
    optimizer.zero_grad()
    x, y = batch_generator(i, X_train, y_train)
    output, hidden = net(x,hidden)
    loss = criterion(output, y)
    hidden = hidden.detach()
    loss.backward()
    nn.utils.clip_grad_norm_(parameters=net.parameters(), max_norm = 10)
    optimizer.step()
    train_loss += loss.item()
    # train_loss_list.append(loss.item())

    if i%1000 == 0 and i != 0:
      train_loss_list.append(train_loss/1000)
      net.eval()     
      #validation
      val_loss = 0
      x, y = batch_generator(j%val_batch_num, X_val, y_val)
      # x, y = batch_generator(100, X_val, y_val)
      output, _ = net(x, hidden)
      output = output.to(device)
      loss = criterion(output, y)
      print('Epoch: ',epoch,'iteration: ',i,',average train loss: ',round(train_loss/1000,4),
              ', val loss: ',round(loss.item(),4))

      train_loss = 0
      j += 1

Epoch:  1 iteration:  1000 ,average train loss:  2.6318 , val loss:  2.8936
Epoch:  1 iteration:  2000 ,average train loss:  2.2962 , val loss:  2.0381
Epoch:  1 iteration:  3000 ,average train loss:  2.1399 , val loss:  2.2235
Epoch:  1 iteration:  4000 ,average train loss:  2.0904 , val loss:  2.4517
Epoch:  1 iteration:  5000 ,average train loss:  1.9992 , val loss:  2.2644
Epoch:  1 iteration:  6000 ,average train loss:  2.0145 , val loss:  1.7624
Epoch:  1 iteration:  7000 ,average train loss:  1.8947 , val loss:  1.9578
Epoch:  1 iteration:  8000 ,average train loss:  1.8159 , val loss:  2.2722
Epoch:  1 iteration:  9000 ,average train loss:  1.8646 , val loss:  1.8051
Epoch:  1 iteration:  10000 ,average train loss:  1.8868 , val loss:  1.9198
Epoch:  1 iteration:  11000 ,average train loss:  1.8305 , val loss:  1.7081
Epoch:  1 iteration:  12000 ,average train loss:  1.8115 , val loss:  1.283
Epoch:  1 iteration:  13000 ,average train loss:  1.8304 , val loss:  1.9235
Epoch:  1

KeyboardInterrupt: ignored

In [ ]:
fig = plt.figure(figsize=(7,7))
ax1 = fig.add_subplot(111)
ax1.plot(range(len(train_loss_list)),train_loss_list,label='train loss')
ax1.legend()
ax1.set_title('Loss Plot')

Produce new sentence

In [ ]:
i = int(val_batch_num*np.random.random(1))

hidden = torch.zeros(num_layers, batch_size, hidden_dim, requires_grad=True).to(device)
x, y = batch_generator(i, X_val, y_val)
output, hidden = net(x, hidden)

train=[]
for id in y:
  train.append(char_index[int(id)])

predict=[]
for id in torch.argmax(output,axis=1):
    predict.append(char_index[int(id)])
''.join(train), ''.join(predict)